In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from navigator_py.brave_api import BraveRequest, BraveResult

brave = BraveResult(BraveRequest(), "does walmart sell handguns")
brave.top(3)

[{'title': 'Walmart Statement on Firearms Policy',
  'url': 'https://corporate.walmart.com/news/2018/02/28/walmart-statement-on-firearms-policy',
  'is_source_local': False,
  'is_source_both': False,
  'description': 'In light of recent events, we’ve taken an opportunity to review our policy on firearm sales. Going forward, we are raising the age restriction for purchase of firearms and ammunition to 21 years of age. We will update our processes as quickly as possible to implement this change.',
  'page_age': '2018-02-28T23:10:09',
  'profile': {'name': 'Walmart',
   'url': 'https://corporate.walmart.com/news/2018/02/28/walmart-statement-on-firearms-policy',
   'long_name': 'corporate.walmart.com',
   'img': 'https://imgs.search.brave.com/MhH6VlRGIN78KbSFiBW3iNIZ4mfNA_OYIQdHk-mMgig/rs:fit:32:32:1:0/g:ce/aHR0cDovL2Zhdmlj/b25zLnNlYXJjaC5i/cmF2ZS5jb20vaWNv/bnMvYWMwZDRhZjIy/MGJlZjc4OWZhYWJh/OThmMGJjYTc1Nzc5/NzVjOTUwZTU4ZjE1/ZGY2Yzg2Yjc1N2Fm/NjdmOTIwMC9jb3Jw/b3JhdGUud2FsbWFy/dC5jb20v'},
  

In [3]:
brave.get_links()

[{'Walmart Statement on Firearms Policy': 'https://corporate.walmart.com/news/2018/02/28/walmart-statement-on-firearms-policy'},
 {'What is Walmart doing to guarantee responsible firearm sales?': 'https://corporate.walmart.com/askwalmart/what-is-walmart-doing-to-guarantee-responsible-firearm-sales'},
 {'r/NJGuns on Reddit: Had no idea Walmart still sells gun products': 'https://www.reddit.com/r/NJGuns/comments/18zhh76/had_no_idea_walmart_still_sells_gun_products/'}]

In [4]:
import requests
from bs4 import BeautifulSoup
import re

# regex to find any whitespace
whitespace = re.compile(r"\s+")


# replace all whitespace with a single space
def clean_whitespace(text):
    return whitespace.sub(" ", text)


links = [v for dictionary in brave.get_links(10) for k, v in dictionary.items()]
soups = [BeautifulSoup(requests.get(link).text, "html.parser") for link in links]
titles = [soup.title.string for soup in soups]
texts = [
    "\n".join(
        text
        for text in [
            clean_whitespace(p.get_text().replace("\n", "").strip())
            for p in soup.find_all("p")
        ]
        if (
            (text.lower().find("enable js") == -1)
            and (text.lower().find("javascript") == -1)
            and (text.lower().find("ad blocker") == -1)
            and (text.lower().find("something went wrong") == -1)
            and (len(text) > 1)
        )
    )
    for soup in soups
]

texts

['Feb. 28, 2018\nFebruary 28, 2018\nIn light of recent events, we’ve taken an opportunity to review our policy on firearm sales. Going forward, we are raising the age restriction for purchase of firearms and ammunition to 21 years of age. We will update our processes as quickly as possible to implement this change.\nIn 2015, Walmart ended sales of modern sporting rifles, including the AR-15. We also do not sell handguns, except in Alaska where we feel we should continue to offer them to our customers. Additionally, we do not sell bump stocks, high-capacity magazines and similar accessories. We have a process to monitor our eCommerce marketplace and ensure our policies are applied.\nWe take seriously our obligation to be a responsible seller of firearms and go beyond Federal law by requiring customers to pass a background check before purchasing any firearm. The law would allow the sale of a firearm if no response to a background check request has been received within three business day

In [12]:
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation

search_results = []
for tt in texts:
    temp = WordPunctTokenizer().tokenize(tt)
    temp = [word.lower() for word in temp]
    temp = [word for word in temp if word not in stopwords.words("english")]
    temp = [
        word for word in temp if word not in punctuation and word not in ["’", "“", "”"]
    ]
    temp = [WordNetLemmatizer().lemmatize(word) for word in temp]

    search_results.append(" ".join(temp))

    print(len(temp))
    print()
    print(" ".join(temp))
    print()

125

feb 28 2018 february 28 2018 light recent event taken opportunity review policy firearm sale going forward raising age restriction purchase firearm ammunition 21 year age update process quickly possible implement change 2015 walmart ended sale modern sporting rifle including ar 15 also sell handgun except alaska feel continue offer customer additionally sell bump stock high capacity magazine similar accessory process monitor ecommerce marketplace ensure policy applied take seriously obligation responsible seller firearm go beyond federal law requiring customer pas background check purchasing firearm law would allow sale firearm response background check request received within three business day policy prohibits sale approval given also removing item website resembling assault style rifle including nonlethal airsoft gun toy heritage company always serving sportsman hunter continue responsible way

111

understand responsibility come deeply rooted place america influence world larg

In [6]:
from navigator_py.generative_ai_provider import OpenAIProvider
from navigator_py.generative_ai_request import OpenAIRequest

In [9]:
oai = OpenAIRequest(OpenAIProvider())

['feb 28 2018 february 28 2018 light recent event taken opportunity review policy firearm sale going forward raising age restriction purchase firearm ammunition 21 year age update process quickly possible implement change 2015 walmart ended sale modern sporting rifle including ar 15 also sell handgun except alaska feel continue offer customer additionally sell bump stock high capacity magazine similar accessory process monitor ecommerce marketplace ensure policy applied take seriously obligation responsible seller firearm go beyond federal law requiring customer pas background check purchasing firearm law would allow sale firearm response background check request received within three business day policy prohibits sale approval given also removing item website resembling assault style rifle including nonlethal airsoft gun toy heritage company always serving sportsman hunter continue responsible way',
 'understand responsibility come deeply rooted place america influence world largest r

In [23]:
system_prompt = """You are a diligent underwriting assistant whose sole job is to determine whether or not a prospective business sells firearms or not. You are tasked with visiting the website of a business and reading the information on the website to determine if they sell firearms. You are not allowed to use any other sources of information other than the website itself. You are not allowed to use any tools or software to help you determine if the business sells firearms. You are only allowed to use your own judgement based on the information on the website. 

Read the following text retrieved from a web search of "does walmart sell handguns" and write a summary of the information you found. Keep in mind that the text has been preprocessed, converting all words to lowercase, removing stopwords, lemmatizing the words, and removing most punctuation.

====================

"""

for i, sr in enumerate(search_results):
    system_prompt += (
        f"Search result {i+1}:"
        + "\n"
        + "=" * len(f"Search result {i+1}")
        + "\n\n"
        + sr
        + "\n\n"
    )

print(system_prompt)

You are a diligent underwriting assistant whose sole job is to determine whether or not a prospective business sells firearms or not. You are tasked with visiting the website of a business and reading the information on the website to determine if they sell firearms. You are not allowed to use any other sources of information other than the website itself. You are not allowed to use any tools or software to help you determine if the business sells firearms. You are only allowed to use your own judgement based on the information on the website. 

Read the following text retrieved from a web search of "does walmart sell handguns" and write a summary of the information you found. Keep in mind that the text has been preprocessed, converting all words to lowercase, removing stopwords, lemmatizing the words, and removing most punctuation.


Search result 1:

feb 28 2018 february 28 2018 light recent event taken opportunity review policy firearm sale going forward raising age restriction purc

In [27]:
post_user_prompt = "Based on the information above, answer yes, no, or 'I cannot tell' to the question: Does walmart sell firearms?. Please provide a brief explanation of your answer, and indicate which sources you used to make your determination."

In [28]:
oai = OpenAIRequest(
    OpenAIProvider(), _system_prompt=system_prompt, _post_user_prompt=post_user_prompt
)
res = oai.prompt("Does Walmart sell firearms?")
res.content

"**Yes, Walmart sells firearms.**\n\n**Explanation:**\nThe search results provide multiple pieces of evidence indicating that Walmart continues to sell firearms, though with several restrictions and changes over time:\n\n1. **Search results 1 and 2**: Both mention that Walmart discontinued the sale of modern sporting rifles (including AR-15s) and handguns (except in Alaska). They also raised the minimum age to purchase firearms and ammunition to 21 years and stopped selling certain types of ammunition. However, they continue to sell firearms meant for hunting and sport shooting.\n\n2. **Search results 5, 8, and 10**: These results confirm that Walmart still sells long-barrel deer rifles and shotguns and describes Walmart as a major seller of firearms and ammunition, though they have scaled back their offerings. Additionally, Walmart's CEO has stated that they aim to balance the needs of hunting and sport shooting enthusiasts with safety concerns.\n\nTherefore, based on the provided inf